# Машинное обучение ранжированию с помощью библиотеки CatBoost

В этом примере мы:
- увидим как выглядят датасеты для машинного обучения ранжированию, на примере стандартного датасета [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)
- познакомимся с библиотекой **CatBoost**
- научимся решать задачу машинного обучения ранжирования используя алгоритмы, реализованные в **CatBoost**

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [ ]:
import copy
from timeit import default_timer as timer

import catboost
from catboost import datasets, utils

## Датасет MSLR (Microsoft Learning to Rank)

Дальше мы будем работать с датасетом MSLR.

Полный датасет можно скачать с официального сайта: https://www.microsoft.com/en-us/research/project/mslr/

Мы этого делать не будем т.к. в CatBoost уже встроена возможность загрузить небольшой сабсет MSLR, с которым мы и будем работать дальше.

Загрузим этот сабсет:

In [ ]:
df_train, df_test = datasets.msrank_10k()

Датасет представляет собой обычный табличный датасет из 130 колонок:
- В первой колонке лежит таргет (оценка асессора), по 5-балльной шкале релевантности: от 0 до 4 (включительно)
- Во второй колонке лежит ID запроса, по которому можно сгруппировать все оценки документов в рамках одного и того же запроса
- Дальше идет вектор из 128 фичей (таких как значения BM25 и т.п.), их точная природа нам сейчас на важна

Посмотрим на данные:

In [ ]:
print(df_train.head(5))

Для удобства присвоим колонкам говорящие имена:

In [ ]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns

# Assign column names
columns = generate_column_names(num_features=136)
df_train.columns = columns
df_test.columns = columns

Теперь наши данные выглядят красивее:

In [ ]:
print(df_train.head(5))

Проведем небольшой EDA.

Всего у нас 10000 документов в трейне:

In [ ]:
print(df_train.info())

И 10000 документов в тесте:

In [ ]:
print(df_test.info())

Сколько у нас запросов?

In [ ]:
num_queries_train = df_train['qid'].nunique()
num_queries_test = df_test['qid'].nunique()
print(f"Got {num_queries_train} train and {num_queries_test} test queries")

Получается, у нас примерно по 100 документов на запрос.

Это типично, когда, например, для сбора датасета обкачивались и заливались на оценку топ-100 документов поисковой выдачи по случайным запросам.

Теперь посмотрим на распределение таргетов (оценок):

In [ ]:
print(df_train['label'].value_counts())

In [ ]:
print(df_test['label'].value_counts())

Теперь нам надо представить датасет в формате, который можно подавать на вход катбустовой модели.  
Для этого придется разделить его на 3 части:

- **y** -- вектор таргетов
- **X** -- тензор из фичей
- **q** -- вектор из ID запросов, которые позволяют сгруппировать все документы, которые относятся к одному и тому же запросу

CatBoost требует, чтобы в векторе **q** одинаковые ID запроса шли подряд (но в отличие от, например, xgboost, не требует их строгой сортированности). Однако в нашем случае никаких дополнительных действий не потребуется т.к. датасет уже и так отсортирован по qid.

Убедимся в этом:

In [ ]:
print(df_train['qid'].is_monotonic_increasing)
print(df_test['qid'].is_monotonic_increasing)

Сконвертируем датасет в нужный формат:

In [ ]:
def to_catboost_dataset(df):
    y = df['label'].to_numpy()                       # Label: [0-4]
    q = df['qid'].to_numpy().astype('uint32')        # Query Id
    X = df.drop(columns=['label', 'qid']).to_numpy() # 136 features
    return (X, y, q)

X_train, y_train, q_train = to_catboost_dataset(df_train)
X_test, y_test, q_test = to_catboost_dataset(df_test)

## Обучаем pointwise модель

Теперь можно приступить непосредственно к обучению модели. Мы начнем с простой pointwise модели которая в качестве лосса использует обычное RMSE.

Подготовим пулы катбуста:

In [ ]:
pool_train = catboost.Pool(data=X_train, label=y_train, group_id=q_train)
pool_test = catboost.Pool(data=X_test, label=y_test, group_id=q_test)

Зададим целевую метрику, которую будем оптимизировать.  
В нашем случае будем использовать NDCG@10:

In [ ]:
EVAL_METRIC = 'NDCG:top=10;type=Exp'

Подготовим параметры обучения модели, в т.ч.:
- целевую метрику
- сид генератора случайных чисел
- число итераций, после которого останавливаем обучение если в течение данного числа итераций мы не наблюдаем улучшения целевой метрики на валидационном множестве

Если хотим обучаться на GPU, то еще надо добавить параметр *task_type=GPU*

In [ ]:
DEFAULT_PARAMS = {
    'iterations': 1000,            # maximum possible number of trees
    'early_stopping_rounds': 100,  # stop if metric does not improve for N rounds
    'eval_metric': EVAL_METRIC,    # # metric used for early stopping
    'random_seed': 22,
    'verbose': 10
}

Мы будем обучать разные модели, использующие разные лоссы, соответствующие разным алгоритмам машинного обучения ранжированию.  
Напишем функцию, которая позволит кастомизировать модель под нужный лосс:

In [ ]:
def create_model(loss_function):
    params = copy.deepcopy(DEFAULT_PARAMS)

    # Temporary directory that is used by catboost to store additional information
    catboost_info_dir = f"/tmp/catboost_info.{loss_function.lower()}"

    params.update({
        'loss_function': loss_function,
        'train_dir': str(catboost_info_dir),
    })
    return catboost.CatBoost(params)

Создадим модель:

In [ ]:
model = create_model('RMSE')
print(model)

И зафитим ее на нашем обучающем множестве.

Количество деревьев будет выбрано автоматически с использованием т.н. early stopping -- процесс обучения будем остановлен после того, как на валидационном множестве перестанет расти наша целевая метрика (т.е. NDCG).

Для этого передадим в функцию fit() в качестве валидационного множества (параметр eval_set) наш тест-сет.

ВНИМАНИЕ: строго говоря, так делать нельзя т.к. приведет к переобучению. По хорошему, мы должны были сначала разбить наш трейн на собственно обучающее и валидационное множества, и передавать в eval_set уже это валидационное множества. А тест-сет надо было сохранить и использовать уже только в самом конце для подсчета финальных скоров. Однако, для простоты, мы так делать не будем, и оставим все это в качестве упражнения.

In [ ]:
# Fit
start = timer()
model.fit(pool_train, eval_set=pool_test, use_best_model=True)
elapsed = timer() - start
print(f"Model fit: num_trees = {model.tree_count_} elapsed = {elapsed:.3f}")

Видим, что модель состоит из 239 деревьев, и лучший скор NDCG@10 на тесте равен **0.419**

При желании, мы теперь можем сохранить модель в формате cbm:

In [ ]:
model_file = "/tmp/model.cbm"

# Save model
model.save_model(model_file)

# Load model
# model = catboost.CatBoost()
# model.load_model(model_file)

Получим предикты модели на тестовом множестве:

In [ ]:
y_hat_test = model.predict(pool_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Теперь, имея предикты, можно посчитать метрики качества:

In [ ]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp', 'DCG:top=10;type=Exp', 'MAP:top=10']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

Мы видим, что значение NDCG@10 на тесте совпало с тем, что вывел сам катбуст во время обучения модели!

## Обучаем YetiRank

Теперь проделаем все то же самое, но на этот раз с использованием алгоритма YetiRank:

In [ ]:
# Create model
model = create_model('YetiRank')

# Fit
start = timer()
model.fit(pool_train, eval_set=pool_test, use_best_model=True)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f} num_trees = {model.tree_count_}")

# Predict
y_hat_test = model.predict(pool_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

# Compute metrics on test
print("\nEvaluated:")
compute_metrics(y_test, y_hat_test, q_test)

Видно, что теперь модель обучается значительно дольше.

Сравним результаты:

- RMSE модель выбила NDCG@10 = 0.419
- а YetiRank выбивает уже NDCG@10 = 0.439!

Таким образом мы наглядно видим преимущество pairwise/listwise-подхода над "наивным" pointwise-подходом.